<a href="https://colab.research.google.com/github/samina-if/AdventOfCode2024/blob/main/Advent_of_Code_Day24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required library
import re

def parse_input(file_path):
    """Parses the input file into initial wire values and gate instructions."""
    with open(file_path, 'r') as f:
        data = f.read().strip()

    # Split the data into sections
    sections = data.split('\n\n')
    initial_values = {}
    instructions = []

    # Parse initial wire values
    for line in sections[0].split('\n'):
        wire, value = line.split(': ')
        initial_values[wire] = int(value)

    # Parse gate instructions
    for line in sections[1].split('\n'):
        instructions.append(line)

    return initial_values, instructions

def evaluate_gate(op, val1, val2):
    """Evaluates a gate operation."""
    if op == "AND":
        return val1 & val2
    elif op == "OR":
        return val1 | val2
    elif op == "XOR":
        return val1 ^ val2
    else:
        raise ValueError(f"Unknown operation: {op}")

def simulate(initial_values, instructions):
    """Simulates the logic gate system and computes the output."""
    wires = initial_values.copy()

    # Parse and execute each instruction
    while instructions:
        remaining_instructions = []
        for instruction in instructions:
            match = re.match(r"(\w+) (AND|OR|XOR) (\w+) -> (\w+)", instruction)
            if match:
                inp1, op, inp2, out = match.groups()
                if inp1 in wires and inp2 in wires:
                    wires[out] = evaluate_gate(op, wires[inp1], wires[inp2])
                else:
                    remaining_instructions.append(instruction)
            else:
                raise ValueError(f"Invalid instruction format: {instruction}")
        instructions = remaining_instructions

    # Collect binary output from wires starting with 'z'
    binary_output = "".join(
        str(wires[f"z{str(i).zfill(2)}"]) for i in range(len(wires)) if f"z{str(i).zfill(2)}" in wires
    )

    # Ensure binary_output has leading zeros for all relevant wires
    binary_values = [
        str(wires[f"z{str(i).zfill(2)}"]) for i in range(len(wires)) if f"z{str(i).zfill(2)}" in wires
    ]
    binary_output = "".join(binary_values[::-1])

    # Convert binary output to decimal
    decimal_output = int(binary_output, 2)

    return binary_output, decimal_output

if __name__ == "__main__":
    # Read input and simulate
    input_file = "input.txt"
    initial_values, instructions = parse_input(input_file)
    binary_result, decimal_result = simulate(initial_values, instructions)
    print("Output in binary:", binary_result)
    print("Output in decimal:", decimal_result)


Output in binary: 1010010001111100001111101001111011111011101000
Output in decimal: 45213383376616


In [ ]:
# Install necessary packages
!pip install z3-solver

import z3
import random
from itertools import product, combinations
from collections import defaultdict, deque
from functools import cache

# Utility function to split the input into lines
def lines(s: str):
    return s.strip().split('\n')

# Read from input.txt
with open("input.txt", "r") as f:
    A, B = f.read().split("\n\n")

# Dictionary to store wire values
G = dict()

# Process the first part of the input (A) to fill G
for l in lines(A):
    a, b = l.split(": ")
    G[a] = int(b)

# Dictionary to store operations
ops = {}

# Process the second part of the input (B) to fill ops
for l in lines(B):
    x, dest = l.split(" -> ")
    a, op, b = x.split()
    ops[dest] = (a, op, b)

# The z wires are sorted in descending order based on their number
zs = {s for s in ops if s[0] == "z"}
zs = sorted(zs, key=lambda x: int(x[1:]), reverse=True)

# Function to simulate the circuit
def sim(G):
    n = len(zs)
    i = 0
    while i < n:
        for d, (a, op, b) in ops.items():
            if d in G:
                continue
            if a in G and b in G:
                x, y = G[a], G[b]
                if op == "AND":
                    G[d] = x & y
                elif op == "OR":
                    G[d] = x | y
                elif op == "XOR":
                    G[d] = x ^ y
                else:
                    assert False

                if d in zs:
                    i += 1

    return int("".join(str(G[z]) for z in zs), 2)

# Create adjacency list
def mkadj():
    adj = {s: [a, b] for s, (a, _, b) in ops.items()}
    for s in G:
        adj[s] = []
    return adj

# Check if the graph has cycles
def is_cyclic():
    return topsort(mkadj())[1]

# Breadth-first search (BFS) to find connected components
def bfs(adj, start):
    dist = {start: 0}
    prev = {start: None}
    q = deque([start])

    while q:
        u = q.popleft()
        for v in adj[u]:
            if v not in dist:
                dist[v] = dist[u] + 1
                prev[v] = u
                q.append(v)

    return dist, prev

# Topological sorting
def topsort(adj):
    indeg = defaultdict(int)
    for u in adj:
        for v in adj[u]:
            indeg[v] += 1

    q = deque()
    for u in adj:
        if indeg[u] == 0:
            q.append(u)

    order = []
    while q:
        u = q.popleft()
        order.append(u)
        for v in adj[u]:
            indeg[v] -= 1
            if indeg[v] == 0:
                q.append(v)

    return order, len(order) != len(adj)

# Check if a wire can be swapped
def swappable(s: str):
    return set(bfs(mkadj(), s)[1]) - set(G)

# Generate test cases for validation
@cache
def testf(i: int):
    DIFF = 6
    if i < DIFF:
        tests = list(product(range(1 << i), repeat=2))
    else:
        tests = []
        for _ in range(1 << (2 * DIFF)):
            a = random.randrange(1 << i)
            b = random.randrange(1 << i)
            tests.append((a, b))

    random.shuffle(tests)
    return tests

# Function to check if the condition holds for a specific set of wires
def f(i: int, swapped: set[str]):
    if i == 46:
        res = ",".join(sorted(swapped))
        print(f"Answer: {res}")
        return

    # Function to calculate the value of a wire
    def getv(s: str, a: int, b: int) -> int:
        if s[0] == "x":
            return (a >> int(s[1:])) & 1
        if s[0] == "y":
            return (b >> int(s[1:])) & 1
        av, op, bv = ops[s]
        x, y = getv(av, a, b), getv(bv, a, b)
        if op == "AND":
            return x & y
        if op == "OR":
            return x | y
        if op == "XOR":
            return x ^ y

    # Function to check if all tests pass
    def check():
        for a, b in testf(i):
            for j in range(i + 1):
                x = getv(f"z{j:02}", a, b)
                if x != ((a + b) >> j) & 1:
                    return False
        return True

    works = check()
    print(i, works, swapped)
    if works:
        f(i + 1, swapped)
        return

    if len(swapped) == 8:
        return

    inside = swappable(f"z{i:02}") - swapped
    outside = set(ops) - swapped
    to_test = list(product(inside, outside)) + list(combinations(inside, 2))
    random.shuffle(to_test)

    for a, b in to_test:
        if a == b:
            continue
        ops[a], ops[b] = ops[b], ops[a]
        swapped.add(a)
        swapped.add(b)
        if not is_cyclic() and check():
            f(i, swapped)
        swapped.remove(a)
        swapped.remove(b)
        ops[a], ops[b] = ops[b], ops[a]

# Start the search
f(0, set())
